In [1]:
import pandas as pd
from fuzzywuzzy import process

In [2]:
df_gc = pd.read_csv('../data/usga_scrdb_aug_2024.csv')

In [16]:
gc_states = df_gc['State'].unique().tolist()
gc_states_str = [item for item in gc_states if isinstance(item, str)]
gc_states_str = sorted(gc_states_str)

In [39]:
df_gc[['Course Name','City','State']].drop_duplicates().head()

,Course Name,City,State
0,Querencia,San Jose del Cabo,MEX
15,Real Del Mar Golf Resort,Rosarito,MEX
23,El Dorado Golf & Beach Club,San Jose del Cabo,MEX
30,Cabo del Sol,Cabo San Lucas,MEX
56,Cabo Real Golf Course,San Jose Del Cabo,MEX


In [56]:
df_gc_unique = df_gc[['Course Name','City','State']].drop_duplicates()

# extract lists from df
gc_states = df_gc['State'].unique().tolist()
courses = sorted(df_gc['Course Name'].unique().tolist())
crs_unsorted = df_gc['Course Name'].unique().tolist()

In [81]:
comma_courses = [element for element in courses if '--' in element]
comma_courses

[]

In [70]:
query = 'Colorado Golf Club'

In [73]:
matched_courses = process.extract(query, courses, limit=10)
matched_courses = [course[0] for course in matched_courses]
matched_courses

['Colorado Golf Club',
 'Coronado Golf Club',
 'Coorow Golf Club',
 'Cooroy Golf Club',
 'Cradoc Golf Club',
 'Dorado Golf Club',
 '052\tMEDAL COUNTRY CLUB PILAR',
 '058\tN.S. DE GUADALUPE CAMPO DE GOLF',
 '060\tESTANCIA VILLA MARIA GOLF CLUB',
 '061 CHACRAS LA PRIMAVERA GOLF']

In [63]:
df_gc_unique[df_gc_unique['Course Name'] == 'Colorado Golf Club']

,Course Name,City,State
12978,Colorado Golf Club,Parker,CO


In [77]:
df_result = df_gc_unique[df_gc_unique['Course Name'].isin(matched_courses)]

In [78]:
# Create the list of strings: Course Name, City, State
result_course_list = df_result.apply(lambda row: f"{row['Course Name']}, {row['City']}, {row['State']}", axis=1).tolist()
result_course_list

['Colorado Golf Club, Parker, CO',
 'Coronado Golf Club, Coronado, CA',
 'Dorado Golf Club, Tucson, AZ',
 'Coorow Golf Club, coorow, QLD',
 'Cradoc Golf Club, Powys, POW',
 'Cooroy Golf Club, Cooroy, QLD',
 '060\tESTANCIA VILLA MARIA GOLF CLUB, MAXIMO PAZ, B',
 '052\tMEDAL COUNTRY CLUB PILAR, PILAR, B',
 '058\tN.S. DE GUADALUPE CAMPO DE GOLF, VILLA ADELINA, B',
 '061 CHACRAS LA PRIMAVERA GOLF, CARLOS KEEN, B']

In [3]:
course_str = 'Cherry Hills Country Club -- Cherry Hills Village -- CO' # 'Colorado Golf Club -- Parker-- CO'
course_name, city, state = [item.strip() for item in course_str.split('--')]
print(course_name, city, state)

Cherry Hills Country Club Cherry Hills Village CO


In [7]:
df_gc[df_gc['Course Name'] == 'Cherry Hills Country Club']

,Course Name,City,State,Tee Name,Gender,Par,Course Rating™,Bogey Rating™,Slope Rating®,RatingF9,...,Front (9),Back (9),Bogey Rating (F9),Bogey Rating (B9),Slope (F9),Slope (B9),TeeID,Length,usga_course_id,timestamp
12990,Cherry Hills Country Club,Cherry Hills Village,CO,Red,M,72,75.5,102.3,145,36.2,...,36.2 / 139,39.3 / 150,49.1,53.2,139.0,139.0,293450,7316.0,1900,2024-08-09 09:45:28.602021
12991,Cherry Hills Country Club,Cherry Hills Village,CO,Black,M,72,73.5,100.4,144,35.6,...,35.6 / 133,37.9 / 155,48.0,52.4,133.0,133.0,293451,7016.0,1900,2024-08-09 09:45:28.602021
12992,Cherry Hills Country Club,Cherry Hills Village,CO,Blue (Member),M,72,72.3,98.7,142,35.2,...,35.2 / 131,37.1 / 153,47.4,51.3,131.0,131.0,293452,6694.0,1900,2024-08-09 09:45:28.602021
12993,Cherry Hills Country Club,Cherry Hills Village,CO,White,M,72,70.2,95.4,136,33.9,...,33.9 / 128,36.3 / 143,45.8,49.6,128.0,128.0,291733,6278.0,1900,2024-08-09 09:45:28.602021
12994,Cherry Hills Country Club,Cherry Hills Village,CO,Green,M,72,67.1,90.0,123,32.7,...,32.7 / 119,34.4 / 127,43.8,46.2,119.0,119.0,293404,5573.0,1900,2024-08-09 09:45:28.602021
12995,Cherry Hills Country Club,Cherry Hills Village,CO,Gold,M,72,60.0,78.1,98,29.7,...,29.7 / 98,30.3 / 97,38.8,39.3,98.0,98.0,435161,4034.0,1900,2024-08-09 09:45:28.602021
12996,Cherry Hills Country Club,Cherry Hills Village,CO,White,F,73,76.4,111.3,148,37.1,...,37.1 / 143,39.3 / 153,54.0,57.3,143.0,143.0,291736,6278.0,1900,2024-08-09 09:45:28.602021
12997,Cherry Hills Country Club,Cherry Hills Village,CO,Green/White,F,73,73.6,107.9,146,36.2,...,36.2 / 142,37.4 / 149,52.9,55.0,142.0,142.0,293449,5763.0,1900,2024-08-09 09:45:28.602021
12998,Cherry Hills Country Club,Cherry Hills Village,CO,Green,F,73,72.5,106.4,144,35.5,...,35.5 / 138,37.0 / 149,51.8,54.6,138.0,138.0,293433,5573.0,1900,2024-08-09 09:45:28.602021
12999,Cherry Hills Country Club,Cherry Hills Village,CO,Short Green,F,73,72.2,105.7,142,35.3,...,35.3 / 136,36.9 / 148,51.3,54.4,136.0,136.0,498191,5532.0,1900,2024-08-09 09:45:28.602021


In [6]:
filtered_df = df_gc[
    (df_gc['Course Name'] == course_name) &
    (df_gc['City'] == city) &
    (df_gc['State'] == state)
].reset_index(drop=True)
filtered_df = filtered_df[['Tee Name','Length','Gender']].drop_duplicates()
filtered_df

,Tee Name,Length,Gender
0,Red,7316.0,M
1,Black,7016.0,M
2,Blue (Member),6694.0,M
3,White,6278.0,M
4,Green,5573.0,M
5,Gold,4034.0,M
6,White,6278.0,F
7,Green/White,5763.0,F
8,Green,5573.0,F
9,Short Green,5532.0,F


In [9]:
filtered_df.apply(lambda row: f"{row['Tee Name']} -- {round(row['Length'])} -- {row['Gender']}", axis=1).tolist()

['Red -- 7316 -- M',
 'Black -- 7016 -- M',
 'Blue (Member) -- 6694 -- M',
 'White -- 6278 -- M',
 'Green -- 5573 -- M',
 'Gold -- 4034 -- M',
 'White -- 6278 -- F',
 'Green/White -- 5763 -- F',
 'Green -- 5573 -- F',
 'Short Green -- 5532 -- F',
 'Gold -- 4034 -- F']

## Calc Score Differentials

### One at a time

In [ ]:
def calc_score_diff(adjusted_gross_score, course_rating, slope_rating, pcc=0):
    """
    Calculate the score differential for a round of golf.

    Parameters:
    adjusted_gross_score (float): The adjusted gross score for the round.
    course_rating (float): The course rating.
    slope_rating (int): The slope rating.
    pcc (int, optional): The Playing Conditions Calculation adjustment. Default is 0.

    Returns:
    float: The score differential.
    """
    score_differential = (113 / slope_rating) * (adjusted_gross_score - course_rating - pcc)

    return score_differential

In [ ]:
# Example usage:
adjusted_gross_score = 85
course_rating = 72.0
slope_rating = 130
pcc = 0  # Adjust if there's a PCC adjustment

differential = calc_score_diff(adjusted_gross_score, course_rating, slope_rating, pcc)
print(f"Score Differential: {differential:.2f}")

### Multiple at once

In [ ]:
def calc_handicap_index(adjusted_gross_scores, course_ratings, slope_ratings, pcc_adjustments=None):
    """
    Calculate the USGA Handicap Index for a player.

    Parameters:
    adjusted_gross_scores (list of float): List of adjusted gross scores for each round.
    course_ratings (list of float): List of course ratings corresponding to each round.
    slope_ratings (list of int): List of slope ratings corresponding to each round.
    pcc_adjustments (list of int, optional): List of PCC adjustments for each round. Default is None.

    Returns:
    float: The player's Handicap Index.
    """
    if pcc_adjustments is None:
        pcc_adjustments = [0] * len(adjusted_gross_scores)

    score_differentials = []
    for i in range(len(adjusted_gross_scores)):
        differential = (113 / slope_ratings[i]) * (adjusted_gross_scores[i] - course_ratings[i] - pcc_adjustments[i])
        score_differentials.append(differential)

    # Sort the score differentials and select the lowest 8 if there are at least 20 rounds
    score_differentials.sort()
    num_differentials = min(8, len(score_differentials))
    best_differentials = score_differentials[:num_differentials]

    # Calculate the average of the selected differentials and apply the 0.96 multiplier
    if best_differentials:
        average_differential = sum(best_differentials) / num_differentials
        handicap_index = average_differential * 0.96
    else:
        handicap_index = None

    return handicap_index

In [ ]:
# Example usage:
adjusted_gross_scores = [85, 90, 88, 92, 87, 86, 84, 89, 91, 85]
course_ratings = [72.0, 71.5, 72.3, 73.0, 71.8, 72.5, 72.0, 71.9, 72.2, 72.4]
slope_ratings = [130, 128, 132, 135, 129, 131, 130, 127, 133, 134]
pcc_adjustments = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # Adjust if needed

handicap_index = calc_handicap_index(adjusted_gross_scores, course_ratings, slope_ratings, pcc_adjustments)
print(f"Handicap Index: {handicap_index:.2f}")

### Take in Scores, Course info and Return Handicap

In [3]:
def calculate_handicap_index_from_courses(df_gc, scores, course_info):
    """
    Calculate the USGA Handicap Index for a player based on course information.

    Parameters:
    df_gc (pd.DataFrame): DataFrame containing course information.
    scores (list of float): List of adjusted gross scores for each round.
    course_info (list of tuple): List of tuples containing course name, city, state, tee name, and gender.

    Returns:
    float: The player's Handicap Index.
    """
    score_differentials = []

    for i, (course_name, city, state, tee_name, gender) in enumerate(course_info):
        # Look up course rating and slope rating from the DataFrame
        course_data = df_gc[(df_gc['Course Name'] == course_name) &
                            (df_gc['City'] == city) &
                            (df_gc['State'] == state) &
                            (df_gc['Tee Name'] == tee_name) &
                            (df_gc['Gender'] == gender) &
                            (df_gc['Back (9)'] != '/')]

        if not course_data.empty:
            course_rating = course_data.iloc[0]['Course Rating™']
            slope_rating = course_data.iloc[0]['Slope Rating®']

            # Calculate score differential
            differential = (113 / slope_rating) * (scores[i] - course_rating)
            score_differentials.append(differential)
        else:
            print(f"Course information not found for: {course_name}, {city}, {state}, {tee_name}")

    # Sort the score differentials and select the lowest 8 if there are at least 20 rounds
    score_differentials.sort()
    num_differentials = min(8, len(score_differentials))
    best_differentials = score_differentials[:num_differentials]

    # Calculate the average of the selected differentials and apply the 0.96 multiplier
    if best_differentials:
        average_differential = sum(best_differentials) / num_differentials
        handicap_index = average_differential * 0.96
    else:
        handicap_index = None

    return handicap_index

In [18]:
# Example usage:
# Assuming df_gc is your DataFrame with course data
scores = [85, 90, 88, 92, 87, 86, 84, 89, 91, 85]
course_info = [
    ("Colorado Golf Club", "Parker", "CO", "Championship", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Champ/Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Black/Paintbrush", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Paintbrush", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Championship", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Champ/Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Black/Paintbrush", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Paintbrush", "M")
]

handicap_index = calculate_handicap_index_from_courses(df_gc, scores, course_info)
print(f"Handicap Index: {handicap_index:.2f}")

Handicap Index: 8.56


In [7]:
scores = [100,92,95,100,88,94,86,97,88,100,89,98]
course_info = [
    ("Bear Creek Golf Club", "Denver", "CO", "Gold", "M"),
    ("Bear Dance Golf Club", "Larkspur", "CO", "Blue/Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Paintbrush", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Paintbrush", "M"),
    ("Applewood Golf Course", "Golden", "CO", "White", "M"),
    ("Banff Springs", "Banff", "AB", "Black/White", "M"),
    ("Applewood Golf Course", "Golden", "CO", "Blue", "M"),
    ("Bear Creek Golf Club", "Denver", "CO", "Gold", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Paintbrush", "M"),
    ("Mount Massive Golf Course", "Leadville", "CO", "Blue/Black", "M"),
    ("Dos Rios Golf Club", "Gunnison", "CO", "Blue", "M"),
    ("Bear Creek Golf Club", "Denver", "CO", "Gold", "M"),
]

handicap_index = calculate_handicap_index_from_courses(df_gc, scores, course_info)
print(f"Handicap Index: {handicap_index:.2f}")

Handicap Index: 16.72
